## Do all necessary imports and client setup 
###### -- use it to setup the python environment and install all packages
###### --  make sure to start the kernal post installation


In [1]:
# ! pip install yahoo_fin
# ! pip install html5lib
# ! pip install yahooquery
# ! pip install yfinance
# ! pip install langchain
# ! pip install langchain-openai
# ! pip install langchain-community
# ! pip install langfuse


In [ ]:
import os
from langchain_openai import AzureChatOpenAI
# setup azure openai 
llm_call = AzureChatOpenAI(api_key= os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint= os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("OPENAI_API_VERSION"),
        model=os.getenv("MODEL"),
        )

In [ ]:
from uuid import uuid4
from langfuse import Langfuse
from langfuse import get_client
from langfuse.langchain import CallbackHandler

# setup langfuse
LANGFUSE_TRACING_ENABLED = True
Langfuse()
langfuse = get_client()
langfuse_handler = CallbackHandler()

## Get stock code for the provided company name

In [4]:
from yahooquery import search
def get_yahoo_ticker(company_name :str) -> str:
    """
    Extract the stock symbol for a given company name using yahooquery.search.
    """
    result = search(company_name)
    try:
        if 'quotes' in result and result['quotes']:
            top_result = result['quotes'][0]
            # print("get_yahoo_ticker run is successful",top_result['symbol'])
            return top_result['symbol']
        else:
            pass
    except Exception as e:
        print(f"Error getting stock symbol: {e}")
        return None

## Get financial news for the stock code

In [5]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
def get_financial_news(ticker):
    """
    Fetches the complete financial news for a given company stock code using YahooFinanceNewsTool.
    """
    try:
        news_tool = YahooFinanceNewsTool()
        news = news_tool.invoke(ticker)
        return news
    except Exception as e:
        print(f"Error getting news for the company code: {e}")
        return None

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Define pydantic output parser

In [6]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel

class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: list[str]
    places_names: list[str]
    other_companies_referred: list[str]
    related_industries: list[str]
    market_implications: str
    confidence_score: float

output_parser = PydanticOutputParser(pydantic_object=SentimentOutput)

## Register prompt with Langfuse

In [9]:
def _ensure_prompt(name: str, 
                   prompt: str, 
                   model=os.getenv("MODEL"), 
                   temperature=0.7,
                   type="text",
                   labels = "production",
                   update_existing=False):
    """
    Ensure that a prompt with the given name exists in Langfuse.
    If it does not exist, create it with the specified parameters.
    """
    try:
        if update_existing:
            langfuse.create_prompt(name = name, 
                                   prompt = prompt, 
                                   config={"model": model, "temperature": temperature},
                                   labels=[labels], type=type,
                                   output_parser = output_parser.get_format_instructions(),
                                   )
        else:
            langfuse.get_prompt(name)
    except Exception as e:
        print("exception in _ensure_prompt:", e)
    langfuse.create_prompt(
        name=name,
        prompt=prompt,
        config={"model": model, "temperature": temperature},
        labels=[labels],
        type=type,
        output_parser = output_parser.get_format_instructions(),  
    )


def register_prompts():
    """
    Register or update the necessary prompts in Langfuse.
    """
    _ensure_prompt(
        "analyse-market-sentiment",
        """
        You are a financial analyst. Given recent news headlines, analyze the sentiment and extract the following in JSON format:

        {format_instructions}

        News:
        {news}

        Company Name: {company_name}
        Stock Code: {stock_code}
        Make sure to format your response as JSON using the following schema:\n{format_instructions}
        """)

## pass inputs to the prompt fetched from langfuse to use

In [10]:
from langchain_core.prompts import ChatPromptTemplate
fetch_prompt = langfuse.get_prompt("analyse-market-sentiment")
filled_prompt = ChatPromptTemplate.from_messages([
    ('system', fetch_prompt.get_langchain_prompt())
]).partial(format_instructions=output_parser.get_format_instructions())

## define runnables and chain with langfuse tracing

In [ ]:
from langchain_core.runnables import RunnableMap, RunnableLambda

# Create a LangChain chain that takes a company name, fetches its ticker, retrieves financial news, and prepares a prompt for sentiment analysis

# Define a function to map company_name to ticker, news, and prompt variables
def prepare_prompt_inputs(inputs):
    company_name = inputs["company_name"]
    ticker = get_yahoo_ticker(company_name)
    # print(f"Ticker for {company_name}: {ticker}")
    news_text = get_financial_news(ticker)
    # print(f"Financial news for {company_name} ({ticker}): {news_text}")
    return {
        "company_name": company_name,
        "stock_code": ticker,
        "news": news_text
    }

# Wrap the function as a RunnableLambda
prepare_prompt_inputs_runnable = RunnableLambda(prepare_prompt_inputs)

# Compose the chain: company_name -> prepare_prompt_inputs -> prompt -> model -> parser
chain= prepare_prompt_inputs_runnable | filled_prompt | llm_call



## Call chain

In [13]:
# Example usage:
company_name = "Microsoft Corporation"

# Set trace attributes dynamically via enclosing span

with langfuse.start_as_current_span(name="analyse-market-sentiment") as span:
    span.update_trace(
        user_id=str(uuid4()),
        session_id="random-session",
        tags=["assignment-1"],
        input={
            "company_name": company_name
        },
    )

    response = chain.invoke({"company_name": company_name},
                                config={"callbacks":[langfuse_handler]})

    span.update_trace(output={"response": response.content})